In [ ]:
!pip install pfp-api-client
!pip install pandas tqdm matplotlib seaborn optuna sella scikit-learn torch torch_dftd dscribe

In [3]:
# Reaction Network Exploration + NEB + ML Filtering for ZnO-Ketone System

import numpy as np
import matplotlib.pyplot as plt
from ase.io import read, write
from ase.neb import NEB
from ase.optimize import BFGS
from ase.vibrations import Vibrations
from dscribe.descriptors import SOAP
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.preprocessing import StandardScaler
import os



In [4]:
# Step 1: Load IS/FS structures for ZnO-ketone reaction
IS = read("initial.traj")  # ZnO surface + adsorbed ketone
FS = read("final.traj")    # reaction product adsorbed state

# Step 2: Interpolate NEB images
n_images = 7
images = [IS]
for i in range(n_images - 2):
    image = IS.copy()
    image.set_positions(IS.get_positions() + (i + 1)/(n_images - 1)*(FS.get_positions() - IS.get_positions()))
    images.append(image)
images.append(FS)

neb = NEB(images)
neb.interpolate(method='idpp')

# Step 3: Optimize the NEB path
optimizer = BFGS(neb, trajectory="neb.traj")
optimizer.run(fmax=0.05)

# Step 4: Analyze NEB result
energies = [image.get_potential_energy() for image in images]
reaction_coordinate = range(len(images))
plt.plot(reaction_coordinate, energies, marker='o')
plt.xlabel('Reaction Coordinate')
plt.ylabel('Energy (eV)')
plt.title('NEB Pathway (ZnO-Ketone)')
plt.savefig('neb_pathway.png')
plt.show()

# Step 5: Compute SOAP descriptors
from ase import neighborlist
from ase.io import Trajectory
from ase.build import bulk

# Define SOAP descriptor generator
soap = SOAP(species=["Zn", "O", "C", "H"],
            rcut=5.0, nmax=8, lmax=6, sigma=0.3, periodic=True)

X_train_structures = [read("initial.traj"), read("final.traj")]
y_train = [read("initial.traj").get_potential_energy(), read("final.traj").get_potential_energy()]
X_train = soap.create(X_train_structures)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

gpr = GaussianProcessRegressor().fit(X_scaled, y_train)

# Step 6: Predict and screen new paths (placeholder structures)
X_test_structures = [read("test1.traj"), read("test2.traj")]  # hypothetical reaction intermediates
X_test = soap.create(X_test_structures)
X_test_scaled = scaler.transform(X_test)

y_pred, y_std = gpr.predict(X_test_scaled, return_std=True)
print("Predicted barriers:", y_pred)
print("Prediction std dev:", y_std)

# Step 7: Select viable reactions
barrier_threshold = 1.0
uncertainty_threshold = 0.3
selected_idx = [i for i, (y, s) in enumerate(zip(y_pred, y_std)) if y < barrier_threshold and s < uncertainty_threshold]
print("Selected reactions for NEB:", selected_idx)

# Step 8: Future loop over reactions
def run_neb(IS, FS):
    images = [IS]
    for i in range(n_images - 2):
        image = IS.copy()
        image.set_positions(IS.get_positions() + (i + 1)/(n_images - 1)*(FS.get_positions() - IS.get_positions()))
        images.append(image)
    images.append(FS)
    neb = NEB(images)
    neb.interpolate(method='idpp')
    optimizer = BFGS(neb, trajectory="neb_loop.traj")
    optimizer.run(fmax=0.05)
    return neb.get_barrier(), neb

# Example usage:
# for idx in selected_idx:
#     IS = X_test_structures[idx]  # replace with real IS
#     FS = ...  # get corresponding FS
#     barrier, neb_result = run_neb(IS, FS)


FileNotFoundError: [Errno 2] No such file or directory: 'initial.traj'